In [2]:
from transformers.utils import is_cython_available

print("Cython is installed:", is_cython_available())

Cython is installed: True


In [3]:
import pandas as pd

br_edges = pd.read_csv("Edge/totaledge_edge_breast-cancer.csv")
c_edges = pd.read_csv("Edge/totaledge_edge_cancer.csv")
col_edges = pd.read_csv("Edge/totaledge_edge_colorectal-cancer.csv")
end_edges = pd.read_csv("Edge/totaledge_edge_endometrial-cancer.csv")
gli_edges = pd.read_csv("Edge/totaledge_edge_glioma.csv")
ren_edges = pd.read_csv("Edge/totaledge_edge_renal-cell-carcinoma.csv")

edge_dict = {'breast-cancer':br_edges, 'cancer':c_edges, 'colorectal-cancer':col_edges, 'endometrial-cancer': end_edges, 'glioma':gli_edges, 'renal-cell-carcinoma': ren_edges}

nodes = pd.read_csv("Node/FULL.csv")

In [4]:
# nodes = nodes.drop(nodes[nodes["cancer_type"] != "Breast Cancer"].index)
nodes = nodes.dropna(axis=0)

# nodes
nodes = nodes.reset_index(drop=True)
print(nodes["path"].value_counts())

graph_dict = {}

for cancer_type, edges in edge_dict.items():
    graph_dict[cancer_type] = [[], []]
    for idx, row in edges.iterrows():
      col_list = nodes.columns.tolist()
      if(row["source"] not in col_list or row["target"] not in col_list):
        print("something wrong")
        continue
      if(row["weight"]): # if weight != 0
        graph_dict[cancer_type][0].append(nodes.columns.get_loc(row["source"]))
        graph_dict[cancer_type][1].append(nodes.columns.get_loc(row["target"]))

type_dict = {t:i for i, t in enumerate(nodes["path"].unique())}
type_dict

edge_index = [graph_dict[p] for p in nodes["path"]]
edge_attr = [[[1] for i in range(len(graph_dict[p][0]))] for p in nodes["path"]]

for i, k in enumerate(nodes["path"]):
  nodes["path"][i] = [type_dict[nodes["path"][i]]]

node_feat = [[[x] for x in sublist] for sublist in nodes.iloc[:, :-3].values.tolist()]
num_nodes = [nodes.shape[1] - 3] * len(nodes)
y = nodes["path"]

breast-cancer           4099
cancer                  1156
renal-cell-carcinoma     534
glioma                   530
colorectal-cancer        365
endometrial-cancer       173
Name: path, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
from datasets import Dataset, DatasetDict
custom_datadict = {"node_feat" : node_feat, "num_nodes" : num_nodes, 'y': y, 'edge_index': edge_index, 'edge_attr': edge_attr}
custom_dataset = Dataset.from_dict(custom_datadict)
custom_dataset = DatasetDict({"train": custom_dataset})
custom_dataset

# custom_dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['node_feat', 'num_nodes', 'y', 'edge_index', 'edge_attr'],
        num_rows: 6857
    })
})

In [5]:
custom_dataset["train"].features

{'node_feat': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None),
 'num_nodes': Value(dtype='int64', id=None),
 'y': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'edge_index': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'edge_attr': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [ ]:
from transformers.models.graphormer.collating_graphormer import preprocess_item, GraphormerDataCollator
import torch

dataset_processed = custom_dataset.map(preprocess_item, batched=False, writer_batch_size=20)

# split up training into training + validation
train_ds = dataset_processed['train']
torch.save(train_ds, './full_train.pt')
print(train_ds[0].keys())